# 0: Import modules and read data

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt

import os
import time

import random
from scipy.stats import skew, kurtosis, weibull_min
from scipy.special import gamma

### 0.1 Import functions

In [ ]:
# ---------------------------- Function 1 ----------------------------
def acquire_wsp_ds(file_path, start="1979-01-01", end="2024-12-31"):
    ds = xr.open_dataset(file_path)
    s = (
        ds["windspeeds"]
        .sel(time=slice(start, end))
        .to_series()                     
        .replace(-2e30, np.nan)         
        .sort_index()
    )
    return s

# ---------------------------- Function 2 ----------------------------
def weibull_params_and_wpd(data, air_density=1.225):
    data = data.dropna()
    if len(data) < 10:
        return np.nan, np.nan, np.nan
    else:
        shape_p, _, scale_p = weibull_min.fit(data, floc=0, method="MLE")
        wpd = 0.5 * air_density * (scale_p ** 3) * gamma(1 + 3 / shape_p)
        return shape_p, scale_p, wpd

# ---------------------------- Function 3 ----------------------------
def statistic_value(dataset):
    statis = []
    
    statis.append(np.mean(dataset))
    statis.append(np.std(dataset,axis=0)[0])
   
    statis.append(skew(dataset)[0])

    statis.append(kurtosis(dataset)[0])
    
    shape_p, _, scale_p= weibull_min.fit(dataset, floc=0, method="MLE")
    w_wpd = 0.5 * 1.225 * (scale_p ** 3) * gamma(1 + 3 / shape_p) # wind power density
    
    statis.append(shape_p)
    statis.append(scale_p)
    statis.append(w_wpd)
    
    return statis

### 0.2 Acquire data information

In [ ]:

#station = '031710-99999'
file_path = "hadisd.3.4.2.202501p_19310101-20250201_031710-99999.nc"
wsp = acquire_wsp_ds(file_path)

# Interpolate to hourly data
wsp = wsp.asfreq("h").interpolate(method="time")
df_ip = wsp.rename_axis("time").to_frame("windspeeds").reset_index()

# The length of df_ip is 403248 hours from 1979-01-01 to 2024-12-31
df_ip

,time,windspeeds
0,1979-01-01 00:00:00,4.6
1,1979-01-01 01:00:00,4.1
2,1979-01-01 02:00:00,3.6
3,1979-01-01 03:00:00,3.6
4,1979-01-01 04:00:00,3.1
...,...,...
403243,2024-12-31 19:00:00,12.4
403244,2024-12-31 20:00:00,14.4
403245,2024-12-31 21:00:00,12.9
403246,2024-12-31 22:00:00,12.9


In [ ]:
def get_time_df(df):
    
    time_df = pd.DataFrame({
        'Year': pd.to_datetime(df['time'], format="%Y-%m-%dT%H:%M:%S.%fZ").dt.year,
        'Month': pd.to_datetime(df['time'], format="%Y-%m-%dT%H:%M:%S.%fZ").dt.month,
        'Day': pd.to_datetime(df['time'], format="%Y-%m-%dT%H:%M:%S.%fZ").dt.day,
        'Hour': pd.to_datetime(df['time'], format="%Y-%m-%dT%H:%M:%S.%fZ").dt.hour,
        'Minute': pd.to_datetime(df['time'], format="%Y-%m-%dT%H:%M:%S.%fZ").dt.minute,
        'Second': pd.to_datetime(df['time'], format="%Y-%m-%dT%H:%M:%S.%fZ").dt.second,
    })
    
    return time_df

time_df_new_merged_series = get_time_df(df_ip)
new_merged_series_value = df_ip['windspeeds'].values

# 1: Strategies for sampling experiments

->> Run this section only if you need samples from the experiments.

## 1.1: Strategy 1-Continous sampling

->> This section can be run without running the other strategies.

In [ ]:
# The estimated time for this script to run is approximately 12 hours.

data = df_ip.dropna() 

output_dir = '031710-99999_parameters_under_sequential_experiment' # {output directory} - need to be adjusted by your own settings
os.makedirs(output_dir, exist_ok=True)

yrs_in_total = 12 # {the total years of data} - need to be adjusted by the length of your own data
windows = [720 * i for i in range(1, 12 * yrs_in_total + 1)]

for w in windows:
    start_time = time.time()
    file_path = os.path.join(output_dir, f'df_stats_window_{w//720}_months.csv')

    
    if not os.path.exists(file_path):
        
        max_start = len(data) - w
        if max_start < 1000:
            # If the realization candidates < 1000, calculate all windows
            sample_indices = range(w - 1, len(data))
        else:
            # If the realizations candidates > 1000, randomly sample 1000 window end indices, ensuring fixed sampling (due to np.random.seed)
            np.random.seed(int(w / 720))
            sample_indices = np.random.choice(range(w - 1, len(data)), size=1000, replace=False)
            sample_indices = np.sort(sample_indices)  
            
        means = []
        stds = []
        skews = []
        kurtoses = []
        weibull_shapes = []
        weibull_scales = []
        weibull_wpds = []

        for i in sample_indices:
            window_data = data.iloc[i - w + 1:i + 1]
            means.append(window_data.mean().iloc[0])
            stds.append(window_data.std().iloc[0])
            skews.append(skew(window_data, bias=False)[0])
            kurtoses.append(kurtosis(window_data, bias=False)[0])
            shape, scale, wpd = weibull_params_and_wpd(window_data)
            weibull_shapes.append(shape)
            weibull_scales.append(scale)
            weibull_wpds.append(wpd)

        df_stats = pd.DataFrame({
            'mean': means,
            'std': stds,
            'skew': skews,
            'kurtosis': kurtoses,
            'weibull_shape': weibull_shapes,
            'weibull_scale': weibull_scales,
            'weibull_power_density': weibull_wpds
        })
        df_stats.to_csv(file_path, index=False)
    end_time = time.time()
    print(f"Window {w//720} (unit: months) finished calculating; consuming time: {end_time - start_time:.2f} seconds")

## 1.2 Strategy 2-Random sampling

->> This section can be run without running the other strategies.

#### 2.1 When the gap is 1 month

In [ ]:

data = df_ip.dropna() 

output_dir = '031710-99999_parameters_under_random_experiment' # {output directory} - need to be adjusted by your own settings
os.makedirs(output_dir, exist_ok=True)

yrs_in_total = 12
windows = [720 * i for i in range(1, 12 * yrs_in_total + 1)]
data = df_ip.dropna()

iteration_time = 1000

for w in windows:
    start_time = time.time()
    file_path = os.path.join(output_dir, f'df_stats_window_random_{w//720}_months.csv')
    if not os.path.exists(file_path):
        
        rng = np.random.default_rng(seed=w) # seed set to window size for reproducibility
        random_numbers = rng.integers(low=0, high=len(data), size=(iteration_time, w))
        
        stat_values  = np.empty((iteration_time, 7))
    
        for j in range(iteration_time):
            stat_values[j,:] = statistic_value(data.iloc[random_numbers[j,:]].values)
            
        df = pd.DataFrame(stat_values, columns=['mean','std','skew','kurtosis','weibull_shape',
                 'weibull_scale','weibull_power_density'])
        df.to_csv(file_path, index=False)
        
    end_time = time.time()
    print(f"Window {w//720} (unit: months) finished calculating; consuming time: {end_time - start_time:.2f} seconds")

#### 2.2 When the gap is 10 days

In [ ]:

interval = 24*10 
start_range = 24*30  
end_range = 24*365*6  


iteration_time = 1000
num_values = int((end_range-start_range)/interval + 1)


In [ ]:
new_merged_series_value = df_ip['windspeeds'].values

n = 0
start_time = time.time()

for i in range(start_range, end_range+1, interval):   
    
    n += 1
    
    print(f"No.{(n-1),num_values} file is being processed")
          
    if os.path.exists('../result_iteration_random_{}.csv'.format(n-1)):
        continue
    
    rng = np.random.default_rng(seed=i) 
    
    random_numbers = rng.integers(low=0, high=len(new_merged_series_value), size=(iteration_time, i))
    
    stat_values  = np.empty((iteration_time, 7))
    
    for j in range(iteration_time):
        stat_values[j,:] = statistic_value(new_merged_series_value[random_numbers[j,:]])
    
    
    
    df = pd.DataFrame(stat_values, columns=['Mean','Std','Skewness','Kurtosis','Shape','Scale','Energy density'])
    df.to_csv('../Data/New_stations_data_density_16yr_random/SN50500/result_iteration_{}.csv'.format(n-1), index=False)  # 保存到 CSV 文件


end_time = time.time()


execution_time = end_time - start_time
print("Execution_time: ", execution_time, "seconds")

## 1.3 Strategy 3-Diurnal-retained sampling

->> This section can be run without running the other strategies.

->> Samples are equally selected from four time periods (0-5, 6-11, 12-17, 18-23).

In [ ]:

# Acquire indices of four time periods: 0-5, 6-11, 12-17, 18-23
loc_05 = time_df_new_merged_series.index[(time_df_new_merged_series['Hour'] >= 0) & (time_df_new_merged_series['Hour'] <= 5)].tolist()
loc_611 = time_df_new_merged_series.index[(time_df_new_merged_series['Hour'] >= 6) & (time_df_new_merged_series['Hour'] <= 11)].tolist()
loc_1217 = time_df_new_merged_series.index[(time_df_new_merged_series['Hour'] >= 12) & (time_df_new_merged_series['Hour'] <= 17)].tolist()
loc_1823 = time_df_new_merged_series.index[(time_df_new_merged_series['Hour'] >= 18) & (time_df_new_merged_series['Hour'] <= 23)].tolist()

# Print the lengths of these indices to see whether the four time periods have similar data amounts
print(len(loc_05), len(loc_611), len(loc_1217), len(loc_1823))
mean_loc = (len(loc_05)+ len(loc_611)+ len(loc_1217)+ len(loc_1823))/4

print(len(loc_05)-mean_loc, len(loc_611)-mean_loc, len(loc_1217)-mean_loc, len(loc_1823)-mean_loc)


100812 100812 100812 100812
0.0 0.0 0.0 0.0


In [ ]:
# initialize parameters

interval = 24*10 # The interval is 10 days (10*24 hours)
start_range = 24*30 # Start from 1 month
end_range = 24*365*6  # covering 6 years    

iteration_time = 1000
num_values = int((end_range-start_range)/interval + 1)

print(num_values)

In [ ]:

new_merged_series_value = df_ip['windspeeds'].values

n = 0
start_time = time.time()

for i in range(start_range, end_range+1, interval):
    
    n += 1
    
    print(f"No.{(n-1),num_values} file is being processed")
    
    if os.path.exists('../result_iteration_diurnal_{}.csv'.format(n-1)):        
        continue
    
    random.seed(i)
    
    stat_values  = np.empty((iteration_time, 7))
    num_sel = int(i/4) # the number of data to be selected from each time period
    
    for j in range(iteration_time):
        # select certain number of data from each of the four groups, once per iteration        
        random_selection_05 = random.choices(loc_05, k=num_sel)
        random_selection_611 = random.choices(loc_611, k=num_sel)
        random_selection_1217 = random.choices(loc_1217, k=num_sel)
        random_selection_1823 = random.choices(loc_1823, k=num_sel)

        random_selection = [item for sublist in [random_selection_05, random_selection_611, random_selection_1217, random_selection_1823] for item in sublist]
        
        stat_values[j,:] = statistic_value(new_merged_series_value[random_selection])
    
    
    
    df = pd.DataFrame(stat_values, columns=['Mean','Std','Skewness','Kurtosis','Shape','Scale','Energy density'])
    df.to_csv('../031710-99999_parameters_under_diurnalcycle_experiment/result_iteration_diurnal_{}.csv'.format(n-1), index=False)  



end_time = time.time()

execution_time = end_time - start_time
print("Execution_time: ", execution_time, "seconds")
    

## 1.4: Strategy 4-Seasonality-retained sampling

->> This section can be run without running the other strategies.

->> Samples are equally selected from each month.

In [ ]:
# Acquire indices of 12 months
loc_month = {}
num_month = []
for i in range(1,13):
    loc_month[str(i)] = time_df_new_merged_series.index[time_df_new_merged_series['Month'] == i].tolist()
    num_month.append(len(time_df_new_merged_series.index[time_df_new_merged_series['Month'] == i].tolist()))

print(num_month-np.mean(num_month))


# plot the number differences between each month and the mean value
plt.plot(num_month-np.mean(num_month))

In [ ]:
# initialize parameters

interval = 24*10 
start_range = 24*30 
end_range = 24*365*6  

iteration_time = 1000
num_values = int((end_range-start_range)/interval + 1)

print(num_values)

In [ ]:
new_merged_series_value = df_ip['windspeeds'].values

n = 0
start_time = time.time()

for i in range(start_range, end_range+1, interval):
    
    n += 1
    
    print(f"No.{(n-1),num_values} file is being processed")
    
    if os.path.exists('../result_iteration_seasonality_{}.csv'.format(n-1)):        
        continue
    
    random.seed(i)

    
    stat_values  = np.empty((iteration_time, 7))
    num_sel = int(i/12) 
    
    for j in range(iteration_time):
        
        
        random_selection_1 = random.choices(loc_month['1'], k=num_sel)
        random_selection_2 = random.choices(loc_month['2'], k=num_sel)
        random_selection_3 = random.choices(loc_month['3'], k=num_sel)
        random_selection_4 = random.choices(loc_month['4'], k=num_sel)
        random_selection_5 = random.choices(loc_month['5'], k=num_sel)
        random_selection_6 = random.choices(loc_month['6'], k=num_sel)
        random_selection_7 = random.choices(loc_month['7'], k=num_sel)
        random_selection_8 = random.choices(loc_month['8'], k=num_sel)
        random_selection_9 = random.choices(loc_month['9'], k=num_sel)
        random_selection_10 = random.choices(loc_month['10'], k=num_sel)
        random_selection_11 = random.choices(loc_month['11'], k=num_sel)
        random_selection_12 = random.choices(loc_month['12'], k=num_sel)

        random_selection = [item for sublist in [random_selection_1, random_selection_2, random_selection_3, 
                                                 random_selection_4, random_selection_5, random_selection_6,
                                                 random_selection_7, random_selection_8, random_selection_9,
                                                 random_selection_10, random_selection_11, random_selection_12] for item in sublist]
        
        stat_values[j,:] = statistic_value(new_merged_series_value[random_selection])
    

    df = pd.DataFrame(stat_values, columns=['Mean','Std','Skewness','Kurtosis','Shape','Scale','Energy density'])
    df.to_csv('../031710-99999_parameters_under_seasonality_experiment/result_iteration_seasonality_{}.csv'.format(n-1), index=False) 


end_time = time.time()


execution_time = end_time - start_time
print("Execution_time: ", execution_time, "seconds")
    

# 2: Analyzing and visualizing the results from sampling experiments

### 2.0 import functions for this section

In [ ]:
# ---------------------------- Function 1 ----------------------------
# Function 1： define the function to read data
def get_exp_data(folder_path, file_name_start, num_of_files, start_range, interval):
    
    table_prob = np.full([1000,num_of_files,7],np.nan) # 
    x_scatter = np.full([1000,num_of_files],np.nan) # 

    prob_name = ['Mean','Std','Skewness','Kurtosis','Shape','Scale','Energy density']

    for i in range(num_of_files):

        x_scatter[:,i] = start_range + i*interval                 

        file_name = file_name_start + '_' + str(i) + ".csv"
        file_path = os.path.join(folder_path, file_name)

        df = pd.read_csv(file_path)

        for j in range(7):
            table_prob[:,i,j] = df[prob_name[j]] 
    
    return table_prob, x_scatter


# ---------------------------- Function 2 ----------------------------
# Function 2： define the function to read data
def get_statisc_of_series(new_merged_series_value):

    statisc_totalseries = np.empty([7,7])  # 

    statisc_totalseries[:,0] = statistic_value(new_merged_series_value)*1
    statisc_totalseries[:,1] = statisc_totalseries[:,0]*(1+0.1) # +10%
    statisc_totalseries[:,2] = statisc_totalseries[:,0]*(1-0.1) # -10%
    statisc_totalseries[:,3] = statisc_totalseries[:,0]*(1+0.05) # +5%
    statisc_totalseries[:,4] = statisc_totalseries[:,0]*(1-0.05) # -5%
    statisc_totalseries[:,5] = statisc_totalseries[:,0]*(1+0.02) # +2%
    statisc_totalseries[:,6] = statisc_totalseries[:,0]*(1-0.02) # -2%
    
    
    return statisc_totalseries


# ---------------------------- Function 3 ----------------------------
# Function 3： define the function to read data
def range_confidence_level(dataset, z_value): # 1.645-90%, 1.960-95%, 2.326-98%, 2.576-99%
    mean_dataset = np.mean(dataset)
    std_dataset = np.std(dataset)
    
    confidence_interval = [mean_dataset+z_value*(std_dataset), mean_dataset-z_value*(std_dataset)]
    
    ## Another method to calculate confidence interval
    #confidence_interval = [mean_dataset+z_value*(std_dataset/np.sqrt(len(dataset))), mean_dataset-z_value*(std_dataset/np.sqrt(len(dataset)))]
    
    return confidence_interval


# ---------------------------- Function 4 ----------------------------
# Function 4： 
def get_confidence_level(table_prob):
    value_confidence_level = np.full([np.shape(table_prob)[1], 8, np.shape(table_prob)[2]], np.nan) 

    for i in range(np.shape(table_prob)[1]): # 217 experiments in total
        for j in range(np.shape(table_prob)[2]): # seven parameters in total

            value_confidence_level[i, 0, j] = range_confidence_level(table_prob[:,i,j], 1.645)[0] # 求90%的区间
            value_confidence_level[i, 1, j] = range_confidence_level(table_prob[:,i,j], 1.645)[1] # 求90%的区间

            value_confidence_level[i, 2, j] = range_confidence_level(table_prob[:,i,j], 1.96)[0] # 求95%的区间
            value_confidence_level[i, 3, j] = range_confidence_level(table_prob[:,i,j], 1.96)[1] # 求95%的区间

            value_confidence_level[i, 4, j] = range_confidence_level(table_prob[:,i,j], 2.326)[0] # 求98%的区间
            value_confidence_level[i, 5, j] = range_confidence_level(table_prob[:,i,j], 2.326)[1] # 求98%的区间

            value_confidence_level[i, 6, j] = range_confidence_level(table_prob[:,i,j], 2.576)[0] # 求99%的区间
            value_confidence_level[i, 7, j] = range_confidence_level(table_prob[:,i,j], 2.576)[1] # 求99%的区间
            
    return value_confidence_level

### 2.1 read the data from the sampling experiments

In [ ]:
interval = 24*10 
start_range = 24*30  
end_range = 24*365*6  

iteration_time = 1000

num_of_files = int((end_range-start_range)/interval + 1)

# get the distribution parameters of the total series
statisc_totalseries = get_statisc_of_series(new_merged_series_value)

In [ ]:

# ---------------------------- 1. Acquire data from random experiments ----------------------------
folder_path = "../Data/New_stations_data_density_16yr_random/SN50500" # {file folder path} - need to be adjusted by your own settings

# a. get the distribution parameters of all the experiments
table_random, x_scatter = get_exp_data(folder_path,'result_iteration', num_of_files, start_range, interval)

# b. get the 90%, 95%, 99% confidence level of each experiment
confidence_level_random = get_confidence_level(table_random)


# ---------------------------- 2.Acquire data from diurnal-cycle retained experiments ----------------------------

folder_path = "../Data/New_stations_data_density_16yr_withdiurnalcycle/SN50500"
table_diurnal, _ = get_exp_data(folder_path,'result_iteration', num_of_files, start_range, interval)
confidence_level_diurnal = get_confidence_level(table_diurnal)

# ---------------------------- 3. Acquire data from seasonality retained experiments ----------------------------

folder_path = "../Data/New_stations_data_density_16yr_withseasonality/12_months/SN50500"
table_seasonality, _ = get_exp_data(folder_path,'result_iteration', num_of_files, start_range, interval)
confidence_level_seasonality = get_confidence_level(table_seasonality)

#### 2.1.1 Plot figure 2 in the manuscript

In [ ]:


fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(2.6, 7))
prob_name = ['Mean','Std','Skewness','Kurtosis','Shape','Scale','Energy density']
ylabel_name = ['Mean $(m\ s^{-1})$', 'Std. dev. $(m\ s^{-1})$','Skewness','Kurtosis','Weibull k','Weibull c $(m\ s^{-1})$','Energy density $(W\ m^{-2})$']
ylim_value = {'Mean':[2.,4.],'Std':[2.,4.],'Skewness':[0.,4.],'Kurtosis':[-10,40],'Shape':[1.1,2.1],'Scale':[3.6,4.6],'Energy density':[50,120]}

for i, ax in zip(range(3), axes.flatten()):
    
    
    if ax.get_visible():  
        
        
        if i == 0:
            i_x = 0
            ax.set_title('SN50500')
            ax.set_ylabel(ylabel_name[i_x], fontsize=12)
            
        elif i==1:
            i_x = 5
            ax.set_ylabel(ylabel_name[i_x], fontsize=12)
            
        elif i==2:
            i_x = 6
            ax.set_ylabel('Power density $(W\ m^{-2})$', fontsize=12)
            ax.set_xlabel('Number of data')
           

        # setting x axis as log scale
        ax.set_xscale('log') 

        # showing ±5% uncertainty bands
        ax.axhspan(ymin=statisc_totalseries[i_x,4], ymax=statisc_totalseries[i_x,3], color="#EFF3FF", zorder=0, label='±5% uncertainty') # 5% 
        ax.axhspan(ymin=statisc_totalseries[i_x,6], ymax=statisc_totalseries[i_x,5], color="#B0C4DE", zorder=0, label='±2% uncertainty') # 2%

        ax.set_ylim(ylim_value[prob_name[i_x]])
        
        for j in range(num_of_files+1):

            if j == num_of_files:

                ax.scatter(len(new_merged_series_value), statistic_value(new_merged_series_value)[i_x], marker='*', color='r', s=8, zorder=1)
            else:
                ax.scatter(x_scatter[:,j], table_random[:,j,i_x], marker='*', color='k',s=1, alpha=0.2, zorder=1)



        # 展示90%的范围
        ax.plot(x_scatter[0,:], confidence_level_random[:,0,i_x], color="#E69F00", label='Random', linewidth=3, zorder=3)
        ax.plot(x_scatter[0,:], confidence_level_random[:,1,i_x], color="#E69F00", linewidth=3, zorder=3)
        
        ax.plot(x_scatter[0,:], confidence_level_diurnal[:,0,i_x], color="#A07EB3", linestyle='--',label='Diurnal', linewidth=3, zorder=4)
        ax.plot(x_scatter[0,:], confidence_level_diurnal[:,1,i_x], color="#A07EB3", linestyle='--', linewidth=3, zorder=4)

        ax.plot(x_scatter[0,:], confidence_level_seasonality[:,0,i_x], color="#4E79A7", linestyle=':',label='Seasonal', linewidth=3, zorder=5)
        ax.plot(x_scatter[0,:], confidence_level_seasonality[:,1,i_x], color="#4E79A7", linestyle=':', linewidth=3, zorder=5)        
        
        ax.minorticks_on()
        
        #ax.text(-0.2, 1.1, labels[i], transform=ax.transAxes, fontsize=13, fontweight='bold', va='top', ha='right')
        if i_x == 0:
            ax.legend(frameon=False, loc = 'lower right',fontsize=9)


        
plt.tight_layout()

plt.savefig('../Figures/90CI_SN50500_vertical_m.jpeg', dpi=300)


### 2.2 Calculate the fitted equations 

->> describing the relationship between percent error (Y) and sample size (n)

In [ ]:
# 1. calculate percent_error

percent_error = np.full([num_of_files,2,7],np.nan) # 行为217个实验，第一列为上限的error，第二列为下限的error，7为7个指标
for i in range(7):
    
    percent_error[:,0,i] = (confidence_level_random[:,0,i]-statistic_value(new_merged_series_value)[i])/statistic_value(new_merged_series_value)[i]*100
    percent_error[:,1,i] = (confidence_level_random[:,1,i]-statistic_value(new_merged_series_value)[i])/statistic_value(new_merged_series_value)[i]*100


In [ ]:
def percent_error_func_up(n, a, b):
    return np.exp(a * np.log(n) + b)

def percent_error_func_down(n, a, b):
    return -np.exp(a * np.log(n) + b)

In [ ]:
from scipy.optimize import curve_fit


# 2. calculate coefficient of the fitted equation by using percent_error

percent_error_coefficient = np.full([7,2,2],np.nan)  #The 1st column is coefficient a, the 2nd column is coefficient b, the first dimension is up, the second dimension is down
errorbar_ylabel_name = ['Mean (%)', 'Standard deviation (%)','Skewness (%)','Kurtosis (%)','Weibull k parameter (%)','Weibull c parameter (%)','Energy density (%)']

for i in range(7): # 
    
    popt_up, _ = curve_fit(percent_error_func_up, x_scatter[0,:], percent_error[:,0,i])
    percent_error_coefficient[i,0,0], percent_error_coefficient[i,1,0] = popt_up #a__up, b_up = poptup

    popt_down, _ = curve_fit(percent_error_func_down, x_scatter[0,:], percent_error[:,1,i])
    percent_error_coefficient[i,0,1], percent_error_coefficient[i,1,1] = popt_down #a__up, b_up = popt_down
    
    print(errorbar_ylabel_name[i])
    print(f"Y=exp[{round(percent_error_coefficient[i,0,0],3)}ln(n)+{round(percent_error_coefficient[i,1,0],3)}]")
    print(f"Y=-exp[{round(percent_error_coefficient[i,0,1],3)}ln(n)+{round(percent_error_coefficient[i,1,1],3)}]")

#### 2.2.1 Plot figure 4 in the manuscript (plot the percent_error with fitted curve)

In [ ]:
# initialize parameters (start from 10 days to 16 years)

interval_for_figure = 24*10 
start_range_for_figure = 24*30 
end_range_for_figure = 24*365*16  


num_values = int((end_range_for_figure-start_range_for_figure)/interval_for_figure + 1)

x_exp = []
for i in range(num_values):
    x_exp.append(start_range_for_figure + interval_for_figure*i) 


fitting_value2_up = pd.DataFrame()
fitting_value2_down = pd.DataFrame()


fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(8, 5))

fig.delaxes(axes[1, 3])  


ylim_value = {'Mean':[-10,10],'Std':[-10,10],'Skewness':[-60,60],'Kurtosis':[-360,360],'Shape':[-10,10],'Scale':[-10,10],'Energy density':[-26,26]}
 

for i, ax in zip(range(7), axes.flatten()):
    
    # 第一种选择：看全部的参数
    ax.set_ylabel(errorbar_ylabel_name[i])
    ax.set_xlabel('Number of data')
    ax.set_xscale('log')
    ax.set_ylim(ylim_value[prob_name[i]])
    
    ax.plot(x_exp, percent_error_func_up(x_exp, percent_error_coefficient[i,0,0], percent_error_coefficient[i,1,0]), color='orange',linewidth=2)
    ax.plot(x_exp, percent_error_func_down(x_exp, percent_error_coefficient[i,0,1], percent_error_coefficient[i,1,1]), color='orange',linewidth=2)

    ax.scatter(x_scatter[0,:], percent_error[:,0,i], label='Original Data',marker='o', facecolors='none', edgecolors='k', linewidths=0.5)
    ax.scatter(x_scatter[0,:], percent_error[:,1,i], label='Original Data',marker='^', facecolors='none', edgecolors='k', linewidths=0.5)

    fitting_value2_up[errorbar_ylabel_name[i]] = percent_error_func_up(x_exp, percent_error_coefficient[i,0,0], percent_error_coefficient[i,1,0])
    fitting_value2_down[errorbar_ylabel_name[i]] = percent_error_func_down(x_exp, percent_error_coefficient[i,0,1], percent_error_coefficient[i,1,1])    
    
    ax.grid(True)
    ax.minorticks_on()

           
plt.tight_layout()

plt.savefig('../Figures/hourly_exp_percent_error_all_parameters_16years_SN50500.jpeg', dpi=300)

#### 2.2.2 Plot figure 5 in the manuscript (plot the percent_error with fitted curve)

In [ ]:
# initialize parameters (start from 1 day to 30 days)

interval_for_figure = 24*1 
start_range_for_figure = 24*1 
end_range_for_figure = 24*30  

num_values = int((end_range_for_figure-start_range_for_figure)/interval_for_figure + 1)

x_exp = []
for i in range(num_values):
    x_exp.append(start_range_for_figure+interval_for_figure*i) 

  

fitting_value1_up = pd.DataFrame()
fitting_value1_down = pd.DataFrame()

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(8, 5))
fig.delaxes(axes[1, 3])  

ylim_value = {'Mean':[-40,40],'Std':[-40,40],'Skewness':[-300,300],'Kurtosis':[-2000,2000],'Shape':[-40,40],'Scale':[-40,40],'Energy density':[-100,100]}
errorbar_ylabel_name = ['Mean (%)', 'Standard deviation (%)','Skewness (%)','Kurtosis (%)','Weibull k parameter (%)','Weibull c parameter (%)','Energy density (%)']

for i, ax in zip(range(7), axes.flatten()):
    

    ax.set_ylabel(errorbar_ylabel_name[i])
    ax.set_xlabel('Number of data')
    ax.set_ylim(ylim_value[prob_name[i]])

    
    ax.plot(x_exp, percent_error_func_up(x_exp, percent_error_coefficient[i,0,0], percent_error_coefficient[i,1,0]), color='orange',linewidth=2)
    ax.plot(x_exp, percent_error_func_down(x_exp, percent_error_coefficient[i,0,1], percent_error_coefficient[i,1,1]), color='orange',linewidth=2)

    
    fitting_value1_up[errorbar_ylabel_name[i]] = percent_error_func_up(x_exp, percent_error_coefficient[i,0,0], percent_error_coefficient[i,1,0])
    fitting_value1_down[errorbar_ylabel_name[i]] = percent_error_func_down(x_exp, percent_error_coefficient[i,0,1], percent_error_coefficient[i,1,1])

    
    ax.grid(True)
    ax.minorticks_on()
       
           
plt.tight_layout()

plt.savefig('../Figures/hourly_exp_percent_error_all_parameters_30days_SN50500.jpeg', dpi=300)

#### 2.2.3 Plot figure 7 in the manuscript

In [ ]:
output_dir = '031710-99999_parameters_under_sequential_experiment'
table_continuous = get_exp_data(output_dir,'continous')
confidence_level_continuous = get_confidence_level(table_continuous)
maindataset_parameters = statistic_value(df_ip)
statisc_totalseries = get_statisc_of_series(df_ip)

yrs_in_total = 12


HOURS_PER_YEAR  = 8760.0
HOURS_PER_MONTH = 365.25/12*24.0  # ≈ 730.5

fig, axes = plt.subplots(nrows=7, ncols=1, figsize=(3, 16), sharex=False)

months_list = []
for month in range(1, 12 * yrs_in_total + 1):
    months_list.append(month)

for i, ax in zip(range(7), axes.flatten()):
    if ax.get_visible():

        
        x_hours = np.asarray(months_list) * HOURS_PER_MONTH

        
        ax.set_xscale('log')

        ymin_val_5 = min(statisc_totalseries[i,3], statisc_totalseries[i,4])
        ymax_val_5 = max(statisc_totalseries[i,3], statisc_totalseries[i,4])
        
        
        ymin_val_10 = min(statisc_totalseries[i,1], statisc_totalseries[i,2])
        ymax_val_10 = max(statisc_totalseries[i,1], statisc_totalseries[i,2])
        
        
        
        ymin_val_2 = min(statisc_totalseries[i,5], statisc_totalseries[i,6])
        ymax_val_2 = max(statisc_totalseries[i,5], statisc_totalseries[i,6])
        


        hspan1 = ax.axhspan(ymin=ymin_val_5, ymax=ymax_val_5, color='gray', alpha=0.3, label='±5% uncertainty range')
        hspan2 = ax.axhspan(ymin=ymin_val_10, ymax=ymax_val_10, color='gray', alpha=0.2, label='±10% uncertainty range')
        hspan3 = ax.axhspan(ymin=ymin_val_2, ymax=ymax_val_2, color='gray', alpha=0.5, label='±2% uncertainty range')

        line1, = ax.plot(x_hours, confidence_level_continuous[:,0,i], color='#3A3042', linewidth=1.5, label='90% CI (continuous)')
        ax.plot(  x_hours, confidence_level_continuous[:,1,i], color='#3A3042', linewidth=1.5)
        line2, = ax.plot(x_hours, confidence_level_random[:,0,i],     color='orange', linewidth=1.5, label='90% CI (random)')
        ax.plot(  x_hours, confidence_level_random[:,1,i],            color='orange', linewidth=1.5)

        # Add main dataset parameters as red star
        scatter = ax.scatter(16*HOURS_PER_YEAR, maindataset_parameters[i], marker='*', color='r', s=8, label='Main dataset parameters')

        ax.minorticks_on()
        ax.set_ylabel(ylabel_name[i], fontsize=12)

        if i == 0:
            ax.set_title('031710-99999')

        if i == 6:
            ax.set_xlabel('Number of data')


        to_years    = lambda h: h / HOURS_PER_YEAR
        from_years  = lambda y: y * HOURS_PER_YEAR
        secax = ax.secondary_xaxis('top', functions=(to_years, from_years))

plt.tight_layout()
plt.savefig('../Figures/90CI_differences_031710_vertical_updated.jpeg', dpi=300)


## 2.3 Comparison with ERA5 data

### 2.3.1 read the data from the sampling experiments for ERA5

In [ ]:
file_path_10m = "../Data/ERA5_selectedyr_10m_SN50500.xlsx"
file_path_10m.replace("\\","/")

df_10m = pd.read_excel(file_path_10m)

statisc_totalseries_era5_10m = get_statisc_of_series(df_10m['value'])

In [ ]:
interval = 24*10 
start_range = 24*30  
end_range = 24*365*6  

num_of_files = int((end_range-start_range)/interval + 1)


folder_path = "../Data/ERA5_density_random/10m/SN50500"

table_era5_10m, _ = get_exp_data(folder_path,'result_iteration', num_of_files, start_range, interval)
confidence_level_era5_10m = get_confidence_level(table_era5_10m)

### 2.3.2 Plot figure 6 in the manuscript

In [ ]:
ylim_value = {'Mean':[3.,5.4],'Std':[2.2,3.],'Skewness':[0.,2.],'Kurtosis':[-3.8,8],'Shape':[1.2,2.3],'Scale':[3.6,6],'Energy density':[60,160]}

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(2.6, 7))



for i, ax in zip(range(3), axes.flatten()):
    
    
    if ax.get_visible():  
        

        if i == 0:
            i_x = 0
            ax.set_title('SN50500')
            ax.set_ylabel(ylabel_name[i_x], fontsize=12)
            
        elif i==1:
            i_x = 5
            ax.set_ylabel(ylabel_name[i_x], fontsize=12)
            
        elif i==2:
            i_x = 6
            ax.set_ylabel('Power density $(W\ m^{-2})$', fontsize=12)
            ax.set_xlabel('Number of data')

        ax.set_xscale('log') 


        ax.axhspan(ymin=statisc_totalseries[i_x,4], ymax=statisc_totalseries[i_x,3], color='grey', alpha=0.3) # 5%
        ax.axhspan(ymin=statisc_totalseries[i_x,6], ymax=statisc_totalseries[i_x,5], color='grey', alpha=0.8) # 2%

        ax.axhspan(ymin=statisc_totalseries_era5_10m[i_x,4], ymax=statisc_totalseries_era5_10m[i_x,3], color="#EFF3FF", zorder=0) # 5%
        ax.axhspan(ymin=statisc_totalseries_era5_10m[i_x,6], ymax=statisc_totalseries_era5_10m[i_x,5], color="#B0C4DE", zorder=0) # 2%
    
        ax.set_ylim(ylim_value[prob_name[i_x]])
        for j in range(num_of_files+1):

            if j == num_of_files:
                ax.scatter(len(new_merged_series_value), statistic_value(df_10m['value'])[i_x], marker='*', color='r',s=8) # 显示这个数据集的值
                ax.scatter(len(new_merged_series_value), statistic_value(new_merged_series_value)[i_x], marker='*', color='#373737', s=8)
            else:
                
                ax.scatter(x_scatter[:,j], table_era5_10m[:,j,i_x], marker='*', color='k',s=1, alpha=0.2, zorder=1)



        # 展示90%的范围
        ax.plot(x_scatter[0,:], confidence_level_era5_10m[:,0,i_x], color="#E69F00",label='ERA5 10m', linewidth=2, zorder=3)
        ax.plot(x_scatter[0,:], confidence_level_era5_10m[:,1,i_x], color="#E69F00", linewidth=2, zorder=3)

        ax.plot(x_scatter[0,:], confidence_level_random[:,0,i_x], color='#373737', alpha=0.8 ,label='In-situ', linewidth=2, zorder=4)
        ax.plot(x_scatter[0,:], confidence_level_random[:,1,i_x], color='#373737', alpha=0.8, linewidth=2, zorder=4)

        
        ax.minorticks_on()
        
        if i == 0:
            ax.legend(frameon=False, loc = 'center right',fontsize=9)


        
plt.tight_layout()

plt.savefig('../Figures/ERA5_10m_distribution_parameters_SN50500_vertical_m.jpeg', dpi=300)
